# Inactive

In [6]:
!pip list

Package                            Version
---------------------------------- --------------------
absl-py                            1.4.0
alabaster                          0.7.12
anaconda-client                    1.9.0
anaconda-navigator                 2.1.1
anaconda-project                   0.10.1
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            4.3.1
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              21.2.0
autopep8                           1.5.7
Babel                              2.9.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.4
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 

In [1]:
import base64
import json
import os
import requests


SP_ID = os.environ.get("SPOTIFY_ID")
SP_SECRET = os.environ.get("SPOTIFY_SECRET")



def get_token_client_credentials():
    auth_string = f"{SP_ID}:{SP_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : f"Basic {auth_base64}",
        "Content-Type" : "application/x-www-form-urlencoded"
    }
    
    data = {"grant_type":"client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token


def get_token_authorization():
    auth_string = f"{SP_ID}:{SP_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    print(auth_base64)
    
    url = "https://accounts.spotify.com/authorize"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        "Authorization" : f"Basic {auth_base64}",
    }
    
    data = {
        "grant_type":"authorization_code",
        "client_id" : f"{SP_ID}",
        "response_type" : "code",
        "redirect_uri" : "http://localhost:8888/callback",
        "scope" : "user-read-recently-played"      
    }
    result = requests.get(url, headers=headers, data=data)
    print(result)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token





def get_auth_header(token):
    return {"Authorization" : f"Bearer {token}"}

def get_recently_played(token):
    url = "https://api.spotify.com/v1/me/player/recently-played?limit=50"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = json.loads(result.content)
    return json_result

In [2]:
token = get_token_authorization()
print(token)
# get_recently_played(token)

ZDRmN2E4OGEwYmU2NGFhNTk2MjYxMjQ5YTM0OWY1YTE6NjAzMGY2MjE5NDRmNDVhZmIyYjg2YjZmZjlmMjM5YmY=
<Response [400]>


JSONDecodeError: Expecting value: line 2 column 1 (char 1)

# Get Recently Played

In [3]:
import pandas as pd
from datetime import timedelta

In [4]:
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth

OAuth = SpotifyOAuth(
    client_id = os.environ.get("SPOTIFY_ID"),
    client_secret = os.environ.get("SPOTIFY_SECRET"),
    redirect_uri = "http://example.com",
    scope = "user-read-recently-played"
)

sp = spotipy.Spotify(auth_manager=OAuth)

In [5]:
# original print recently played tracks
results = sp.current_user_recently_played()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], "-", track['name'])

0 N.W.A. - Fuck Tha Police
1 Rage Against The Machine - Killing In The Name
2 Bomb at Track - อำนาจเจริญ
3 TaitosmitH - ยุติ-ธรรม (เพลงประกอบภาพยนตร์ คืนยุติ-ธรรม)
4 Black Flag - Rise Above
5 Descendents - 'Merican
6 BABYMETAL - PA PA YA!!
7 Rage Against The Machine - Bulls On Parade
8 Nas - N.Y. State of Mind
9 The Notorious B.I.G. - Hypnotize - 2007 Remaster
10 Radiohead - No Surprises
11 Phoebe Bridgers - Savior Complex
12 James Blake - Say What You Will
13 Taylor Swift - All Too Well (10 Minute Version) (Taylor's Version) (From The Vault)
14 Rage Against The Machine - Killing In The Name
15 Rina Sawayama - Eye For An Eye (Single from John Wick: Chapter 4 Original Motion Picture Soundtrack)
16 Bomb at Track - หมดเวลาแล้วครับพี่ (Ticking Time Bomb)
17 CMTEN - NEVER MET! - 100 gecs r3mix (feat. Glitch Gum)
18 Sandaru Sathsara - Unstoppable
19 Hans Zimmer - Cornfield Chase
20 YEW - Summertime
21 Phoebe Bridgers - Killer
22 Gary Jules - Mad World
23 Phoebe Bridgers - Savior Complex
24 R

In [6]:
# database build
df_db_dict = {'played_at':[],
                'track_name':[],
                'artists_name':[],
                'album_name':[],
                'track_url':[],
                'context':[],
                'album_info':[],
                'artists_info':[],
                'available_markets':[],
                'disc_number':[],
                'duration_ms':[],
                'duration_time_delta':[],
                'explicit':[],
                'external_ids':[],
                'href':[],
                'id':[],
                'is_local':[],
                'popularity':[],
                'preview_url':[],
                'track_number':[],
                'type':[],
                'uri':[]
                }

for item in results['items']:
    
    # assign items
    track = item['track']  
    played_at = item['played_at']
    context = item['context']
    
    track_name = track['name']
    artists_name = track['artists'][0]['name']
    album_name = track['album']['name']
    track_url = track['external_urls']['spotify']
    album_info = track['album']
    artists_info = track['artists']
    available_markets = track['available_markets']
    disc_number = track['disc_number']
    duration_ms = track['duration_ms']
    duration_time_delta = timedelta(milliseconds=duration_ms)
    explicit = track['explicit']
    external_ids = track['external_ids']
    href = track['href']
    id = track['id']
    is_local = track['is_local']
    popularity = track['popularity']
    preview_url = track['preview_url']
    track_number = track['track_number']
    type = track['type']
    uri = track['uri']
    
    # add to dict
    df_db_dict['played_at'].append(played_at)
    df_db_dict['track_name'].append(track_name)
    df_db_dict['artists_name'].append(artists_name)
    df_db_dict['album_name'].append(album_name)
    df_db_dict['track_url'].append(track_url)
    df_db_dict['context'].append(context)
    df_db_dict['album_info'].append(album_info)
    df_db_dict['artists_info'].append(artists_info)
    df_db_dict['available_markets'].append(available_markets)
    df_db_dict['disc_number'].append(disc_number)
    df_db_dict['duration_ms'].append(duration_ms)
    df_db_dict['duration_time_delta'].append(duration_time_delta)
    df_db_dict['explicit'].append(explicit)
    df_db_dict['external_ids'].append(external_ids)
    df_db_dict['href'].append(href)
    df_db_dict['id'].append(id)
    df_db_dict['is_local'].append(is_local)
    df_db_dict['popularity'].append(popularity)
    df_db_dict['preview_url'].append(preview_url)
    df_db_dict['track_number'].append(track_number)
    df_db_dict['type'].append(type)
    df_db_dict['uri'].append(uri)

In [7]:
pd.DataFrame(df_db_dict)

,played_at,track_name,artists_name,album_name,track_url,context,album_info,artists_info,available_markets,disc_number,...,explicit,external_ids,href,id,is_local,popularity,preview_url,track_number,type,uri
0,2023-11-01T06:03:09.397Z,Fuck Tha Police,N.W.A.,Straight Outta Compton,https://open.spotify.com/track/5n8Aro6j1bEGIy7...,"{'type': 'playlist', 'external_urls': {'spotif...","{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AT, BO, BR, BG, CA, CL, CO, CR, CY, CZ, D...",1,...,True,{'isrc': 'USPO10000873'},https://api.spotify.com/v1/tracks/5n8Aro6j1bEG...,5n8Aro6j1bEGIy7Tpo7FV7,False,73,https://p.scdn.co/mp3-preview/9b8444d806154c97...,2,track,spotify:track:5n8Aro6j1bEGIy7Tpo7FV7
1,2023-11-01T05:57:23.475Z,Killing In The Name,Rage Against The Machine,Rage Against The Machine - XX (20th Anniversar...,https://open.spotify.com/track/59WN2psjkt1tyax...,"{'type': 'playlist', 'external_urls': {'spotif...","{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,...,True,{'isrc': 'USSM11205433'},https://api.spotify.com/v1/tracks/59WN2psjkt1t...,59WN2psjkt1tyaxjspN8fp,False,82,https://p.scdn.co/mp3-preview/998d3604b68754c1...,2,track,spotify:track:59WN2psjkt1tyaxjspN8fp
2,2023-11-01T05:52:09.431Z,อำนาจเจริญ,Bomb at Track,Bomb At Track,https://open.spotify.com/track/7iWs087lukviVtV...,"{'type': 'playlist', 'external_urls': {'spotif...","{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,...,True,{'isrc': 'FRX201642607'},https://api.spotify.com/v1/tracks/7iWs087lukvi...,7iWs087lukviVtV7sLTboi,False,22,https://p.scdn.co/mp3-preview/efb138acbe87cfdd...,1,track,spotify:track:7iWs087lukviVtV7sLTboi
3,2023-11-01T05:49:19.315Z,ยุติ-ธรรม (เพลงประกอบภาพยนตร์ คืนยุติ-ธรรม),TaitosmitH,ไททศมิตร,https://open.spotify.com/track/0rgXoSk2aFDC7yz...,"{'type': 'playlist', 'external_urls': {'spotif...","{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,...,False,{'isrc': 'THG012000067'},https://api.spotify.com/v1/tracks/0rgXoSk2aFDC...,0rgXoSk2aFDC7yz2K0HYg8,False,39,https://p.scdn.co/mp3-preview/b2946386ef7d6433...,4,track,spotify:track:0rgXoSk2aFDC7yz2K0HYg8
4,2023-11-01T05:45:05.518Z,Rise Above,Black Flag,Damaged,https://open.spotify.com/track/4kFfFe38CRVnTsa...,"{'type': 'playlist', 'external_urls': {'spotif...","{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,...,False,{'isrc': 'USA370538438'},https://api.spotify.com/v1/tracks/4kFfFe38CRVn...,4kFfFe38CRVnTsakUTL4E4,False,56,https://p.scdn.co/mp3-preview/f4de8bb58a418d82...,1,track,spotify:track:4kFfFe38CRVnTsakUTL4E4
5,2023-11-01T05:42:38.692Z,'Merican,Descendents,Cool to Be You,https://open.spotify.com/track/2BY2O9OybmxpND0...,"{'type': 'playlist', 'external_urls': {'spotif...","{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,...,False,{'isrc': 'USFW40467204'},https://api.spotify.com/v1/tracks/2BY2O9Oybmxp...,2BY2O9OybmxpND0DqNgx97,False,56,https://p.scdn.co/mp3-preview/dba9f806893854a7...,4,track,spotify:track:2BY2O9OybmxpND0DqNgx97
6,2023-11-01T05:40:46.940Z,PA PA YA!!,BABYMETAL,METAL GALAXY,https://open.spotify.com/track/43hAWgANUMvybp1...,"{'type': 'playlist', 'external_urls': {'spotif...","{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, BO, BR, CA, CL, CO, CR, DO, EC, SV, G...",1,...,False,{'isrc': 'JPTF01907203'},https://api.spotify.com/v1/tracks/43hAWgANUMvy...,43hAWgANUMvybp1zDBMrRf,False,58,https://p.scdn.co/mp3-preview/82e74e18473ff0e9...,10,track,spotify:tr

In [11]:
for i in results:
    print(i)

items
next
cursors
limit
href


In [24]:
for i in results['items'][0]:
    print(i)

track
played_at
context


In [46]:
results['items'][0]['context']

{'type': 'playlist',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1EQpj7X7UK8OOF'},
 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1EQpj7X7UK8OOF',
 'uri': 'spotify:playlist:37i9dQZF1EQpj7X7UK8OOF'}

In [30]:
for i in results['items'][0]['track']:
    print(i)

album
artists
available_markets
disc_number
duration_ms
explicit
external_ids
external_urls
href
id
is_local
name
popularity
preview_url
track_number
type
uri


In [65]:
for i in range(len(results['items'])):
    ms = results['items'][i]['track']['duration_ms']
    td = timedelta(milliseconds=ms)
    print(td)

0:03:19.226000
0:04:55.386000
0:06:03.853000
0:05:12.360000
0:03:49.226000
0:03:34.866000
0:01:03.946000
0:02:45.853000
0:03:56.600000
0:03:48.280000
0:03:59.506000
0:02:00.626000
0:04:17.720000
0:03:19.090000
0:02:18.840000
0:03:01.306000
0:03:40.480000
0:03:32.937000
0:03:15.472000
0:03:04.281000
0:04:18.826000
0:06:06.333000
0:04:39.213000
0:07:19.120000
0:03:57.026000
0:02:42.466000
0:04:38.680000
0:03:25.600000
0:04:35.162000
0:04:25.080000
0:01:02.720000
0:03:32.253000
0:01:57.809000
0:01:46.683000
0:03:42.633000
0:03:28.173000
0:01:34.245000
0:02:48.839000
0:05:11.106000
0:04:23.733000
0:00:57.173000
0:03:02.826000
0:05:24.466000
0:03:09.132000
0:02:57.200000
0:02:47.106000
0:03:09.373000
0:06:15.453000
0:05:26.933000
0:03:40.285000


In [78]:
results['items'][0]['track']['external_urls']['spotify']

'https://open.spotify.com/track/2OoS1aZFAFscH1cUiKlazH'

In [47]:
# df pd.DataFrame sample
nested_dict = {
    'Name': ['John', 'Anna', 'Peter'],
    'Age': [28, 24, 30],
    'Occupation': ['Engineer', 'Doctor', 'Teacher']
}
df = pd.DataFrame(nested_dict)

In [48]:
df

,Name,Age,Occupation
0,John,28,Engineer
1,Anna,24,Doctor
2,Peter,30,Teacher


# Up to Drive

In [13]:
credential_json = pd.read_json("client_secrets.json")
credential_json

,web
auth_provider_x509_cert_url,https://www.googleapis.com/oauth2/v1/certs
auth_uri,https://accounts.google.com/o/oauth2/auth
client_id,391196591976-6m9jj1vf01jqrgttch9oh8ag0tpgbpq5....
client_secret,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
project_id,upload-spotify-data
redirect_uris,[http://example.com/]
token_uri,https://oauth2.googleapis.com/token


In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()           
drive = GoogleDrive(gauth)  

In [4]:
# Create a GoogleDriveFile instance with title 'test.txt'.
file1 = drive.CreateFile({'parents': [{'id': '1ttR1ymJfIYwOMhhiRxP1J7WbxMvNlkd9'}],'title': 'test.txt'})  
# Set content of the file from the given string.
file1.SetContentString('Hello World!') 
file1.Upload()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=391196591976-6m9jj1vf01jqrgttch9oh8ag0tpgbpq5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [9]:
upload_file_list = ['1.jpg', '2.jpg']
for upload_file in upload_file_list:
    gfile = drive.CreateFile({'parents': [{'id': '1ttR1ymJfIYwOMhhiRxP1J7WbxMvNlkd9'}]})
    # Read file and set it as the content of this instance.
    gfile.SetContentFile(upload_file)
    gfile.Upload() # Upload the file.